## Discriminant Analysis



### LDA

Let $x_1, \ldots, x_N$ be observations from $g$ groups. Suppose that
observations in the $k$-th group are normally distributed with mean $\mu_k$
and covariance matrix $\Sigma_k$. The likelihood of observing $x$ given $F_k$
is given by
\begin{eqnarray}
  f (x \mid F_k) & \propto & | \Sigma_k |^{- 1 / 2} \exp \left( - \frac{1}{2}
  (x - \mu_k)^T \Sigma_k^{- 1} (x - \mu_k) \right) p_k.
\end{eqnarray}
where $p_k =$probability of $x$ generated by $F_k$. If we assume homoscedasticity, we can let $\Sigma = \Sigma_k$ and drop the $| \Sigma_k |^{- 1 / 2}$ and 
$x^{T}\Sigma_k^{-1}x$ terms. Therefore
\begin{eqnarray}
  \log (f (F_k \mid x)) & \propto & \log (p_k) - \frac{1}{2} \mu_k^T
  \Sigma^{- 1} \mu_k + \mu_k^T \Sigma^{- 1} x.
\end{eqnarray}

We can estimate $p_k$, $\mu_k$ and $\Sigma$ from the data. In
particular, $\Sigma$ is estimated using the the pooled sample covariance
matrix
\begin{eqnarray}
  S_p & = & \frac{\sum_{i = 1}^g (n_i - 1) S_i}{N - g}
\end{eqnarray}
where $S_i$ is the sample covariance matrix calculated from samples in the
$i$-th group.

Then we classify $x$ to be in the $m$-th group where  $m =
\underset{k}{\mathrm{argmax}} \; \log (f (F_k \mid x))$.


In [1]:
import pandas as pd
import numpy  as np
from numpy.linalg import inv
import math
import scipy.stats

l    = math.log
bart = scipy.stats.bartlett
d    = pd.read_csv('insect.csv')

print bart(d[d['Type']=='a'][['Width1','Width2','Width3']].as_matrix(), d[d['Type']=='b'][['Width1','Width2','Width3']].as_matrix())

mu          = lambda z : np.matrix(d[d['Type']==z][['Width1','Width2','Width3']].mean().as_matrix())
inv_cov     = lambda z : inv(d[d['Type']==z].cov().as_matrix())
size_       = lambda z : len(d[d['Type']==z])
inv_cov_mat = ((size_('a') -1) *inv_cov('a') + (size_('a') -1) *inv_cov('b')) /(size_('a')+size_('b')-2)
lin_score   = lambda z, pred: (l(1/2.)-mu(z)*inv_cov_mat*mu(z).transpose()/2. + mu(z)*inv_cov_mat*np.matrix(pred).transpose())[0,0]

x = np.array([194, 124, 49])

map(lambda z : lin_score(z, x),['a','b'])

BartlettResult(statistic=0.37520119134801999, pvalue=0.54018273368732772)


[203.74386446915341, 206.89183456812958]

### QDA 
If we drop the homoscedasticity assumption, then the score function becomes quadratic since we can no longer drop the $x^{T}\Sigma_k^{-1}x$ term, so we have

\begin{eqnarray}
  \log (f (F_k \mid x)) & \propto & \log (p_k) - \frac{1}{2} \left(\log(\det(\Sigma_k))+
  (x - \mu_k)^T \Sigma_k^{- 1} (x - \mu_k) \right)
\end{eqnarray}

